Exporting models made using this package

Code should be run in a FBL Medulla kernel

In [15]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import networkx
import numpy as np
import os.path
from random import choice
import tempfile
import typing as tp
import flybrainlab as fbl

import linecache
import os
import tracemalloc

import netpyne
from netpyne.specs import netParams, SimConfig
from netpyne import conversion, sim

from brian2 import *

import src.NeuroNLP_to_Brian_Netpyne.nlpToNetpyne as toNet
import src.NeuroNLP_to_Brian_Netpyne.nlptoBrian2 as toBri

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define variables and helper functions

In [24]:
# Query and files
query = "show cholinergic neurons presynaptic to T4a neuron in column C with at least 5 synapses"

# Cells Present: Mi1-C, Mi1-L, Tm3-C-ant, Tm3-K-ant, Tm3-home-post

## NetPyNE

This project leverages NetPyNE to export generated models to both python and LEMS formats.

Generated python files can be imported into other NetPyNE projects, or used in NePyNE's GUI interface (found [here](https://netpyne.v2.opensourcebrain.org/)).

Generated LEMS files can be used in neuroml simulations, or leveraged using neuroml to export to other neural simulators (See [here](https://docs.neuroml.org/Userdocs/SimulatingNeuroMLModels.html) for information on simulating neuroml models). It should be noted that, due to how networks are constructed in NetPyNE, generated models are not functional in Brian2.

In [21]:
# Default Mechanisms
stim_mech = {'bkg': {'type': 'NetStim', 'rate': 10, 'noise': 0}} # Stimulation mechanism
default_syn = {'mod': 'Exp2Syn', 'tau1': 0.1, 'tau2': 5.0, 'e': 0} # Excitatory synapse mechanism

default_cell = {'secs': {}}
default_cell['secs']['soma'] = {'geom': {}, 'mechs': {}}
default_cell['secs']['soma']['geom'] = {'diam': 18.8, 'L': 18.8, 'Ra': 123.0}                           # soma geometry
default_cell['secs']['soma']['mechs']['hh'] = {'gnabar': 0.12, 'gkbar': 0.036, 'gl': 0.003, 'el': -70}  # soma hh mechanism

In [25]:
networkParameters, simConfig = toNet.model_gen(client=my_client,
                                               res=res,
                                               filename=path+"Import_Model",
                                               default_mech=default_syn,
                                               default_cell=default_cell,
                                               stim_sources=stim_mech,
                                               stim_targets={'Mi1_C': {'source': 'bkg', 'weight': 0.5, 'delay': 1, 'mech': 'exc'}},
                                               record_names=['Mi1_C', 'Mi1_L', 'Tm3_K_ant'],
                                               maintain_morphology=False)

# Export to python format
toNet.export_model(networkParams=networkParameters,
                   simConfig=simConfig,
                   filename="sample_python",
                   exp_type="python")

# Export to neuroml format
toNet.export_model(networkParams=networkParameters,
                   simConfig=simConfig,
                   filename="sample_neuroml",
                   exp_type="neuroml")

script saved on sample_python
Exporting the network to NeuroML 2, reference: sample_neuroml, connections: True, stimulations: True, format: xml
Is Tm3_K_ant in odict_keys(['Tm3_K_ant', 'Tm3_C_ant', 'Tm3_home_post', 'Mi1_C', 'Mi1_L'])...?
Proceeding with assumption <netpyne.network.pop.Pop object at 0x00000182A7D75AC8> defines which cellParams...
Now cell params for Tm3_K_ant are: {'secs': {'soma': {'geom': {'diam': 18.8, 'L': 18.8, 'Ra': 123.0}, 'pointps': {'Izhi': {'mod': 'Izhi2007b', 'C': 1, 'k': 0.7, 'vr': -60, 'vt': -40, 'vpeak': 35, 'a': 0.03, 'b': -2, 'c': -50, 'd': 100, 'celltype': 1}}}}, 'conds': {'cellType': 'Tm3_K_ant', 'cellModel': 'Tm3_K_ant'}}...
Assuming abstract cell with behaviour set by single point process: {'mod': 'Izhi2007b', 'C': 1, 'k': 0.7, 'vr': -60, 'vt': -40, 'vpeak': 35, 'a': 0.03, 'b': -2, 'c': -50, 'd': 100, 'celltype': 1}!
Is Tm3_C_ant in odict_keys(['Tm3_K_ant', 'Tm3_C_ant', 'Tm3_home_post', 'Mi1_C', 'Mi1_L'])...?
Proceeding with assumption <netpyne.netwo

## Brian2

Brian2 does not support exporting models to other simulation programs.